In [1]:
import os
import pandas as pd
import matplotlib as mlt
import numpy as np
from scipy.stats import chi2_contingency
import lightgbm as lgb
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [2]:
os.chdir('C:\\Users\\hp\\Desktop\\ml6#House_grade\\Dataset')

In [3]:
import h2o

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.131-b11, mixed mode)
  Starting server from C:\Users\hp\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\hp\AppData\Local\Temp\tmpp309fxj7
  JVM stdout: C:\Users\hp\AppData\Local\Temp\tmpp309fxj7\h2o_hp_started_from_python.out
  JVM stderr: C:\Users\hp\AppData\Local\Temp\tmpp309fxj7\h2o_hp_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.3
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_hp_rkde1q
H2O cluster total nodes:,1
H2O cluster free memory:,1.299 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [5]:
df = h2o.import_file(path="C:\\Users\\hp\\Desktop\\ml6#House_grade\\Dataset\\train_set.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
test_set = h2o.import_file(path="C:\\Users\\hp\\Desktop\\ml6#House_grade\\Dataset\\test_set.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
itrain = df.drop(['building_id'], axis=1)

In [8]:
itest = test_set.drop(['building_id'], axis=1)

In [9]:
damage_predictors = itrain.columns
damage_predictors.remove('damage_grade')

In [10]:
var_target ='damage_grade'

In [21]:
train, valid, test = df.split_frame(ratios=[0.5,0.3], seed = 123 )


In [43]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

rf_1 = H2ORandomForestEstimator( model_id="rf_v1", ntrees = 699, stopping_rounds =2, score_each_iteration=True, mtries = 34,
                                balance_classes = True, seed=100, nbins = 25)

In [44]:
rf_1.train(damage_predictors, var_target, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [45]:
rf_1.confusion_matrix(data=valid)

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Error,Rate
15631.0,1989.0,632.0,98.0,8.0,0.1485456,"2,727 / 18,358"
471.0,12746.0,8441.0,3943.0,49.0,0.5030799,"12,904 / 25,650"
119.0,6280.0,16896.0,13307.0,89.0,0.5395056,"19,795 / 36,691"
12.0,1407.0,6867.0,36995.0,346.0,0.1891862,"8,632 / 45,627"
2.0,42.0,189.0,2445.0,60461.0,0.0424144,"2,678 / 63,139"
16235.0,22464.0,33025.0,56788.0,60953.0,0.2466735,"46,736 / 189,465"


In [32]:
final_rf = rf_1.predict(test)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [36]:
final_rf

predict,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5
Grade 1,0.969979,0.0300209,0,0,0
Grade 2,0.114252,0.468766,0.365568,0.0514143,0
Grade 3,0.148253,0.318242,0.391746,0.140444,0.00131421
Grade 5,0,6.40448e-07,0,0,0.999999
Grade 2,0,0.86267,0.100292,0.0359195,0.00111874
Grade 5,0,0,0,0,1
Grade 5,0,6.40448e-07,0,0,0.999999
Grade 4,0,0.0814648,0.305433,0.571044,0.0420576
Grade 4,0,0.0272295,0.119571,0.84015,0.0130492
Grade 2,0,0.382422,0.362638,0.241237,0.013703


In [29]:
print(rf_1)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_v1


ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.20890604702590687
RMSE: 0.4570624104276208
LogLoss: 0.5937972708334861
Mean Per-Class Error: 0.24342518817565914
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Error,Rate
92087.0,8540.0,4295.0,396.0,6.0,0.1256789,"13,237 / 105,324"
512.0,63001.0,26822.0,15025.0,38.0,0.4022562,"42,397 / 105,398"
106.0,13837.0,54914.0,36504.0,120.0,0.4793944,"50,567 / 105,481"
27.0,2851.0,13886.0,88202.0,299.0,0.1620957,"17,063 / 105,265"
1.0,79.0,358.0,4590.0,100379.0,0.0477008,"5,028 / 105,407"
92733.0,88308.0,100275.0,144717.0,100842.0,0.2434961,"128,292 / 526,875"


Top-5 Hit Ratios: 


k,hit_ratio
1,0.7565039
2,0.9175421
3,0.9885229
4,0.9992218
5,1.0



ModelMetricsMultinomial: drf
** Reported on validation data. **

MSE: 0.21191117407229793
RMSE: 0.4603381084293347
LogLoss: 0.5884243170347278
Mean Per-Class Error: 0.29971909663299856
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2018-07-29 20:47:12,0.000 sec,0.0,nan,nan,nan,nan,nan,nan
,2018-07-29 20:47:24,12.108 sec,1.0,0.4778342,1.3613407,0.2881482,0.4854043,2.2174121,0.2932626
,2018-07-29 20:47:34,21.679 sec,2.0,0.4721332,1.1896456,0.2805074,0.4691374,1.0008355,0.2788167
,2018-07-29 20:47:43,30.824 sec,3.0,0.4693692,1.0843619,0.2748756,0.4659377,0.7548230,0.2730161
,2018-07-29 20:47:52,39.304 sec,4.0,0.4691667,0.9576654,0.2735766,0.4648185,0.6693601,0.2701185
,2018-07-29 20:48:01,48.764 sec,5.0,0.4665405,0.8804666,0.2677372,0.4633857,0.6360252,0.2659119
,2018-07-29 20:48:11,58.272 sec,6.0,0.4647581,0.8164226,0.2646477,0.4626807,0.6184185,0.2648827
,2018-07-29 20:48:20,1 min 7.717 sec,7.0,0.4634293,0.7656544,0.2615437,0.4623488,0.6093061,0.2641491
,2018-07-29 20:48:30,1 min 17.687 sec,8.0,0.4621430,0.7245669,0.2584985,0.4617248,0.6048440,0.2631568
,2018-07-29 20:48:41,1 min 29.001 sec,9.0,0.4601955,0.6969787,0.2545623,0.4611207,0.6003265,0.2619745


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
height_ft_post_eq,654894.6875000,1.0,0.1860660
condition_post_eq_Not damaged,407944.9375000,0.6229169,0.1159036
count_floors_post_eq,398516.4687500,0.6085199,0.1132249
has_repair_started,280537.3125000,0.4283701,0.0797051
area_assesed_Both,205993.8750000,0.3145450,0.0585261
---,---,---,---
plan_configuration_Building with Central Courtyard,82.2517014,0.0001256,0.0000234
plan_configuration_E-shape,65.2671509,0.0000997,0.0000185
plan_configuration_H-shape,48.4799423,0.0000740,0.0000138
has_secondary_use_gov_office,36.8537827,0.0000563,0.0000105



See the whole table with table.as_data_frame()



district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id,ward_id,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,area_assesed_Both,area_assesed_Building removed,area_assesed_Exterior,area_assesed_Interior,area_assesed_Not able to inspect,legal_ownership_status_Institutional,legal_ownership_status_Other,legal_ownership_status_Private,legal_ownership_status_Public,land_surface_condition_Flat,land_surface_condition_Moderate slope,land_surface_condition_Steep slope,foundation_type_Bamboo/Timber,foundation_type_Cement-Stone/Brick,foundation_type_Mud mortar-Stone/Brick,foundation_type_Other,foundation_type_RC,roof_type_Bamboo/Timber-Heavy roof,roof_type_Bamboo/Timber-Light roof,roof_type_RCC/RB/RBC,ground_floor_type_Brick/Stone,ground_floor_type_Mud,ground_floor_type_Other,ground_floor_type_RC,ground_floor_type_Timber,other_floor_type_Not applicable,other_floor_type_RCC/RB/RBC,other_floor_type_TImber/Bamboo-Mud,other_floor_type_Timber-Planck,position_Attached-1 side,position_Attached-2 side,position_Attached-3 side,position_Not attached,plan_configuration_Building with Central Courtyard,plan_configuration_E-shape,plan_configuration_H-shape,plan_configuration_L-shape,plan_configuration_Multi-projected,plan_configuration_Others,plan_configuration_Rectangular,plan_configuration_Square,plan_configuration_T-shape,plan_configuration_U-shape,condition_post_eq_Covered by landslide,condition_post_eq_Damaged-Not used,condition_post_eq_Damaged-Repaired and used,condition_post_eq_Damaged-Rubble Clear-New building built,condition_post_eq_Damaged-Rubble clear,condition_post_eq_Damaged-Rubble unclear,condition_post_eq_Damaged-Used in risk,condition_post_eq_Not damaged,building_id,damage_grade
21,0,0,0,0,0,0,0,0,2,2102,210204,1,0,0,0,0,0,0,0,0,0,0,0,2,2,1,620,12,12,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1e96b58fa58,Grade 1
39,0,0,0,0,0,0,0,0,1,3929,392902,1,0,0,0,0,0,0,0,0,0,0,0,2,2,13,202,17,17,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,392cbd772c9,Grade 3
9,0,0,0,0,0,0,0,0,1,913,91313,1,0,0,0,0,0,0,0,0,0,0,0,1,1,18,455,9,9,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,d49acbd60b,Grade 2
21,1,0,0,0,1,0,0,0,0,2149,214907,0,0,0,0,0,0,0,0,0,0,0,0,3,0,18,348,22,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1f45e8dec7e,Grade 5
12,0,0,0,0,0,0,0,0,0,1230,123005,1,0,0,0,0,0,0,0,0,0,0,0,1,1,22,100,7,7,0,1,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,11e64b184bf,Grade 2
24,0,0,0,0,0,0,0,0,0,2408,240809,1,0,0,0,0,0,0,0,0,0,0,0,2,0,22,392,17,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,230ad59877a,Grade 5
36,1,1,0,0,0,0,0,0,0,3642,364202,1,1,0,0,1,0,0,0